In [1]:
#!pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [1]:
import pandas as pd
df = pd.read_csv('bbc-text.csv')
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [2]:
%run preprocess.ipynb

In [3]:
df['clean_text'] = df['text'].apply(preprocess2)

In [4]:
df.head(15)

,category,text,clean_text
0,tech,tv future in the hands of viewers with home th...,future viewer theatre system plasma digital re...
1,business,worldcom boss left books alone former worldc...,worldcom former worldcom bernie ebbers accused...
2,sport,tigers wary of farrell gamble leicester say ...,tiger farrell gamble leicester rushed making f...
3,sport,yeading face newcastle in fa cup premiership s...,yeading newcastle premiership newcastle united...
4,entertainment,ocean s twelve raids box office ocean s twelve...,twelve office twelve sequel starring george cl...
5,politics,howard hits back at mongrel jibe michael howar...,howard mongrel michael howard leader acting at...
6,politics,blair prepares to name poll date tony blair is...,prepares likely election parliament return eas...
7,sport,henman hopes ended in dubai third seed tim hen...,henman henman slumped straight defeat ljubicic...
8,sport,wilkinson fit to face edinburgh england captai...,wilkinson edinburgh england captain wilkinson ...
9,entertainment,last star wars not for children the sixth an...,child suitable child george minute revenge dar...


In [5]:
df.shape

(2225, 3)

In [6]:
df.to_csv("bbc_preprocessed2.csv")

In [7]:
docs =[]
for d in df.clean_text:
    docs.append(d.split())
type(docs)

list

In [8]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
#dictionary.filter_extremes(no_below=20, no_above=0.5)

In [9]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [10]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 19000
Number of documents: 2225


In [11]:
# Train LSI model.
from gensim.models import LsiModel

model = LsiModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    chunksize=50000,
    )

In [12]:
model.print_topics(5,5)

[(0,
  '0.440*"people" + 0.242*"government" + 0.167*"service" + 0.155*"mobile" + 0.139*"company"'),
 (1,
  '-0.402*"mobile" + 0.325*"government" + 0.277*"labour" + -0.248*"technology" + 0.219*"election"'),
 (2,
  '0.348*"people" + -0.207*"market" + 0.198*"mobile" + -0.197*"england" + -0.194*"company"'),
 (3,
  '-0.358*"labour" + -0.325*"mobile" + -0.322*"election" + 0.251*"people" + 0.193*"forsyth"'),
 (4,
  '0.261*"england" + -0.242*"company" + 0.235*"player" + -0.214*"government" + -0.205*"increase"')]

In [13]:
#len(top_topics)

In [14]:
# Create a new corpus, made of previously unseen documents.
other_texts = [
    ['computer', 'time', 'mobile'],
    ['vote', 'election', 'candidate'],
    ['defeat', 'player', 'cup', 'england','cricket']
]
other_corpus = [dictionary.doc2bow(text) for text in other_texts]
other_corpus

[[(87, 1), (1156, 1)],
 [(456, 1), (1909, 1)],
 [(240, 1), (574, 1), (886, 1), (3517, 1)]]

In [15]:
unseen_doc = other_corpus[2]
vector = model[unseen_doc]  
vector

[(0, 0.14586739387559824),
 (1, -0.07598636625917994),
 (2, -0.3720677280565789),
 (3, 0.06962822455517059),
 (4, 0.5387881739088838)]

In [16]:
for d in other_corpus:
    vector = model[d]
    print(vector)

[(0, 0.22119918181378934), (1, -0.48110788293197204), (2, 0.19917143192519438), (3, -0.2706069059439858), (4, 0.060591677557132656)]
[(0, 0.11970632605166896), (1, 0.23259361069197787), (2, 0.11872526001300002), (3, -0.3400566085450067), (4, 0.1408282176909585)]
[(0, 0.14586739387559824), (1, -0.07598636625917994), (2, -0.3720677280565789), (3, 0.06962822455517059), (4, 0.5387881739088838)]


In [17]:
top = pd.DataFrame()
top['text'] = df['text']
top['category'] = df['category']

In [18]:
import numpy as np

In [19]:
tscore=[[],[],[],[],[]]
for doc in corpus:
    vector = model[doc]
    i=0
    for t in vector:
        tscore[i].append(np.around(t[1],3))
        i=i+1

In [20]:
print(len(tscore[0]))

2225


In [21]:
i=0
for t in tscore:
    top['t'+str(i)] = tscore[i]
    i=i+1

In [22]:
top.shape

(2225, 7)

In [23]:
top.sample(10)

,text,category,t0,t1,t2,t3,t4
1489,unclear future for striker baros liverpool for...,sport,0.813,0.278,-0.900,0.009,0.883
1907,winn-dixie files for bankruptcy us supermarket...,business,1.655,-0.682,-1.518,-0.465,-1.153
1948,uk athletics agrees new kit deal uk athletics ...,sport,0.718,0.097,-0.811,0.029,0.079
316,takeover offer for sunderland fc bob murray c...,business,2.229,-0.197,-2.738,-0.348,-0.620
983,india seeks to boost construction india has cl...,business,4.810,1.766,-2.750,0.201,-2.921
333,ecb holds rates amid growth fears the european...,business,2.549,0.790,-3.511,-1.893,-1.995
2100,us casino tricks face ban in uk controversia...,politics,2.511,0.136,-0.539,1.006,-0.421
1478,former ni minister scott dies former northern ...,politics,2.645,2.251,-1.163,-0.466,1.208
1311,robinson out of six nations england captain ja...,sport,1.735,0.669,-3.834,0.860,4.757
628,volcano drama erupts on bbc one supervolcano ...,entertainment,1.379,-0.200,-1.363,0.334,1.204
